### Chating with an LLM

```{mermaid}

    sequenceDiagram
        actor U as User
        box rgb(255,255,191) ChatGPT
        participant L as LLM
        end
        
        U ->> L: Why is the sky blue?
        L ->> U: The sky appears blue primarily because of a phenomenon called Rayleigh scattering. When su....
        U ->> L: Make it a haiku
        L ->> U: Sunlight scatters wide,<br/>Blue dances through the clear sky,<br/>Day’s bright, azure sigh.

```


- Impression is that we are interacting directly with chosen model (e.g. GPT 4o)​
- But tools like ChatGPT are complicated systems,  using multiple components to manage history, store information about you, filter and moderate content, rewrite your queries, etc. ​
- These systems are closed and highly opaque.


### Hypothetical but possible architecture 

```{mermaid}
sequenceDiagram
  autonumber
  participant U as User
  box rgb(255,255,191) ChatGPT
  participant FE as Web UI
  participant GW as API Gateway
  participant APP as Application Layer
  participant MOD as Moderation
  participant DB as Chat History DB
  participant RET as Retrieval / Tools
  participant MR as Model Router
  participant LLM as LLM Serving
  participant OBS as Observability
  end

  U->>FE: Type prompt & press Enter
  FE->>GW: POST /chat (session token)
  GW->>APP: Forward request

  APP->>DB: Load recent history
  APP->>MOD: Pre-moderation (input)
  MOD-->>APP: OK / transform / block

  alt Input blocked
    APP-->>FE: Return safe/blocked message
    APP->>OBS: Log policy event
  else Input allowed
    opt Retrieval / tools (optional)
      APP->>RET: Retrieve docs / run tools
      RET-->>APP: Snippets / tool outputs
    end

    APP->>MR: Choose target model (plan/flags)
    MR-->>APP: Model endpoint

    APP->>LLM: Prompt = history + user + snippets
    LLM-->>APP: Stream tokens (partial chunks)

    APP->>MOD: Post-moderation (output)
    MOD-->>APP: OK / redact / re-ask

    APP->>DB: Save user msg & model reply
    APP->>OBS: Metrics / traces / usage

    APP-->>GW: Stream response
    GW-->>FE: Stream to UI
    FE-->>U: Render final answer
  end

```



* The actual LLM just a component in larger system 